# Inputs

In [14]:
%reset -f

In [15]:
import numpy as np
import pandas as pd

from transformers import RobertaTokenizer

from pycaret.classification import *

from controller import Controller

In [16]:
c = Controller('i02')

# Load data

In [17]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [18]:
filepath = fr'{c.get_path_data_prepared()}/01_df_train_val_test.pkl'
df_train_val_test = pd.read_pickle(filepath)

df_train = df_train_val_test.query('TRAIN_VAL_TEST_SPLIT == "Train"')
print(f'{df_train.shape=}')
df_val = df_train_val_test.query('TRAIN_VAL_TEST_SPLIT == "Validation"')
print(f'{df_val.shape=}')
df_test = df_train_val_test.query('TRAIN_VAL_TEST_SPLIT == "Test"')
print(f'{df_test.shape=}')

df_train

df_train.shape=(38655, 114)
df_val.shape=(4295, 114)
df_test.shape=(4773, 114)


,ACN_NUM_ACN,TIME_DATE,TIME_1_LOCAL_TIME_OF_DAY,PLACE_LOCALE_REFERENCE,PLACE_1_STATE_REFERENCE,PLACE_2_RELATIVE_POSITION_ANGLE_RADIAL,PLACE_3_RELATIVE_POSITION_DISTANCE_NAUTICAL_MILES,PLACE_4_ALTITUDE_AGL_SINGLE_VALUE,PLACE_5_ALTITUDE_MSL_SINGLE_VALUE,ENVIRONMENT_FLIGHT_CONDITIONS,ENVIRONMENT_1_WEATHER_ELEMENTS_VISIBILITY,ENVIRONMENT_2_WORK_ENVIRONMENT_FACTOR,ENVIRONMENT_3_LIGHT,ENVIRONMENT_4_CEILING,ENVIRONMENT_5_RVR_SINGLE_VALUE,AIRCRAFT_1_ATC_ADVISORY,AIRCRAFT_1_1_AIRCRAFT_OPERATOR,AIRCRAFT_1_2_MAKE_MODEL_NAME,AIRCRAFT_1_3_AIRCRAFT_ZONE,AIRCRAFT_1_4_CREW_SIZE,AIRCRAFT_1_5_OPERATING_UNDER_FAR_PART,AIRCRAFT_1_6_FLIGHT_PLAN,AIRCRAFT_1_7_MISSION,AIRCRAFT_1_8_NAV_IN_USE,AIRCRAFT_1_9_FLIGHT_PHASE,AIRCRAFT_1_10_ROUTE_IN_USE,AIRCRAFT_1_11_AIRSPACE,AIRCRAFT_1_12_MAINTENANCE_STATUS_MAINTENANCE_DEFERRED,AIRCRAFT_1_13_MAINTENANCE_STATUS_RECORDS_COMPLETE,AIRCRAFT_1_14_MAINTENANCE_STATUS_RELEASED_FOR_SERVICE,AIRCRAFT_1_15_MAINTENANCE_STATUS_REQUIRED_CORRECT_DOC_ON_BOARD,AIRCRAFT_1_16_MAINTENANCE_STATUS_MAINTENANCE_TYPE,AIRCRAFT_1_17_MAINTENANCE_STATUS_MAINTENANCE_ITEMS_INVOLVED,AIRCRAFT_1_18_CABIN_LIGHTING,AIRCRAFT_1_19_NUMBER_OF_SEATS_NUMBER,AIRCRAFT_1_20_PASSENGERS_ON_BOARD_NUMBER,AIRCRAFT_1_21_CREW_SIZE_FLIGHT_ATTENDANT_NUMBER_OF_CREW,AIRCRAFT_1_22_AIRSPACE_AUTHORIZATION_PROVIDER_UAS_,AIRCRAFT_1_23_OPERATING_UNDER_WAIVERS_EXEMPTIONS_AUTHORIZATIONS_UAS_,AIRCRAFT_1_24_WAIVERS_EXEMPTIONS_AUTHORIZATIONS_UAS_,AIRCRAFT_1_25_AIRWORTHINESS_CERTIFICATION_UAS_,AIRCRAFT_1_26_WEIGHT_CATEGORY_UAS_,AIRCRAFT_1_27_CONFIGURATION_UAS_,AIRCRAFT_1_28_FLIGHT_OPERATED_AS_UAS_,AIRCRAFT_1_29_FLIGHT_OPERATED_WITH_VISUAL_OBSERVER_UAS_,AIRCRAFT_1_30_CONTROL_MODE_UAS_,AIRCRAFT_1_31_FLYING_IN_NEAR_OVER_UAS_,AIRCRAFT_1_32_PASSENGER_CAPABLE_UAS_,AIRCRAFT_1_33_TYPE_UAS_,AIRCRAFT_1_34_NUMBER_OF_UAS_BEING_CONTROLLED_UAS_,COMPONENT_AIRCRAFT_COMPONENT,COMPONENT_1_MANUFACTURER,COMPONENT_2_AIRCRAFT_REFERENCE,COMPONENT_3_PROBLEM,AIRCRAFT_2_ATC_ADVISORY,AIRCRAFT_2_1_AIRCRAFT_OPERATOR,AIRCRAFT_2_2_MAKE_MODEL_NAME,AIRCRAFT_2_4_CREW_SIZE,AIRCRAFT_2_5_OPERATING_UNDER_FAR_PART,AIRCRAFT_2_6_FLIGHT_PLAN,AIRCRAFT_2_7_MISSION,AIRCRAFT_2_8_NAV_IN_USE,AIRCRAFT_2_9_FLIGHT_PHASE,AIRCRAFT_2_10_ROUTE_IN_USE,AIRCRAFT_2_11_AIRSPACE,AIRCRAFT_2_12_MAINTENANCE_STATUS_MAINTENANCE_DEFERRED,AIRCRAFT_2_14_MAINTENANCE_STATUS_RELEASED_FOR_SERVICE,AIRCRAFT_2_16_MAINTENANCE_STATUS_MAINTENANCE_TYPE,AIRCRAFT_2_17_MAINTENANCE_STATUS_MAINTENANCE_ITEMS_INVOLVED,AIRCRAFT_2_23_OPERATING_UNDER_WAIVERS_EXEMPTIONS_AUTHORIZATIONS_UAS_,AIRCRAFT_2_24_WAIVERS_EXEMPTIONS_AUTHORIZATIONS_UAS_,AIRCRAFT_2_26_WEIGHT_CATEGORY_UAS_,AIRCRAFT_2_27_CONFIGURATION_UAS_,AIRCRAFT_2_28_FLIGHT_OPERATED_AS_UAS_,AIRCRAFT_2_31_FLYING_IN_NEAR_OVER_UAS_,AIRCRAFT_2_34_NUMBER_OF_UAS_BEING_CONTROLLED_UAS_,PERSON_1_LOCATION_OF_PERSON,PERSON_1_1_LOCATION_IN_AIRCRAFT,PERSON_1_2_REPORTER_ORGANIZATION,PERSON_1_3_FUNCTION,PERSON_1_4_QUALIFICATION,PERSON_1_5_EXPERIENCE,PERSON_1_6_CABIN_ACTIVITY,PERSON_1_7_HUMAN_FACTORS,PERSON_1_8_COMMUNICATION_BREAKDOWN,PERSON_1_9_UAS_COMMUNICATION_BREAKDOWN,PERSON_1_10_ASRS_REPORT_NUMBER_ACCESSION_NUMBER,PERSON_2_LOCATION_OF_PERSON,PERSON_2_1_LOCATION_IN_AIRCRAFT,PERSON_2_2_REPORTER_ORGANIZATION,PERSON_2_3_FUNCTION,PERSON_2_4_QUALIFICATION,PERSON_2_5_EXPERIENCE,PERSON_2_6_CABIN_ACTIVITY,PERSON_2_7_HUMAN_FACTORS,PERSON_2_8_COMMUNICATION_BREAKDOWN,PERSON_2_9_UAS_COMMUNICATION_BREAKDOWN,PERSON_2_10_ASRS_REPORT_NUMBER_ACCESSION_NUMBER,EVENTS_ANOMALY,EVENTS_1_MISS_DISTANCE,EVENTS_2_WERE_PASSENGERS_INVOLVED_IN_EVENT,EVENTS_3_DETECTOR,EVENTS_4_WHEN_DETECTED,EVENTS_5_RESULT,ASSESSMENTS_CONTRIBUTING_FACTORS_SITUATIONS,ASSESSMENTS_1_PRIMARY_PROBLEM,REPORT_1_NARRATIVE,REPORT_1_1_CALLBACK,REPORT_2_NARRATIVE,REPORT_2_1_CALLBACK,REPORT_1_2_SYNOPSIS,TRAIN_VAL_TEST_SPLIT,EVENT_RISK,EVENT_RISK_STR
0,1574675,201808,0601-1200,SNA.Airport,CA,NaN,NaN,NaN,5000.0,NA,NA,NA,Daylight,NA,NA,TRACON SCT,Air Carrier,B737-700,NA,2.0,Part 121,IFR,Passenger,FMS Or FMC; GPS,Initial Approach,NA,Class C SNA,NA,NA,NA,NA,NA,NA,NA,NaN,NaN,NaN,NA,NA,N

# Prepare data

In [19]:
features = ['REPORT_1_NARRATIVE', 'REPORT_1_1_CALLBACK', 'REPORT_2_NARRATIVE', 'REPORT_2_1_CALLBACK', 'REPORT_1_2_SYNOPSIS']

## Check missing values

In [20]:
for feature in features:
    missing = df_train.query(f'{feature} == "NA"').shape[0]
    total = df_train.shape[0]
    print(f'{feature}, {missing=}, {total=} => {missing / total:.2%}')

REPORT_1_NARRATIVE, missing=0, total=38655 => 0.00%
REPORT_1_1_CALLBACK, missing=37474, total=38655 => 96.94%
REPORT_2_NARRATIVE, missing=29544, total=38655 => 76.43%
REPORT_2_1_CALLBACK, missing=38592, total=38655 => 99.84%
REPORT_1_2_SYNOPSIS, missing=0, total=38655 => 0.00%


## Get max length for narratives

In [21]:
for feature in features:
    lens = df_train[feature].map(len)
    print(f'{feature} => max={np.max(lens)} avg={np.mean(lens):.2f}')

REPORT_1_NARRATIVE => max=11991 avg=1586.71
REPORT_1_1_CALLBACK => max=3959 avg=20.95
REPORT_2_NARRATIVE => max=11955 avg=228.82
REPORT_2_1_CALLBACK => max=2749 avg=3.04
REPORT_1_2_SYNOPSIS => max=959 avg=171.53


# Model development

Use word embeddings for `REPORT_1_2_SYNOPSIS`.

In [22]:
def preprocess_inputs(df):
    df = df.copy()
    
    X = df['REPORT_1_2_SYNOPSIS']
    y = df['EVENT_RISK']
    
    X = X.apply(lambda x: tokenizer.encode(x, padding='max_length', truncation=True))
    new_df = X.apply(pd.Series)
    new_df['label'] = y
    
    return new_df

In [23]:
df_pycaret = preprocess_inputs(df_val)
df_pycaret

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,label
38655,0,1301,7629,33737,431,41,16031,1068,77,4465,824,4507,160,80,3054,15,10,7329,768,4,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4
38656,0,1779,29738,3030,5,163,38504,12,2965,2524,21,14987,11,10,1881,11159,7,5,16899,6880,7089,13,10,289,3888,13565,381,2371,2336,2892,5,2524,13173,768,81,5,5038,8,156,10,10358,131,19755,6684,23,10,12148,14417,4,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1

In [24]:
clf = setup(data=df_pycaret, target='label', session_id=42, use_gpu=True)

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

,Description,Value
0,Session id,42
1,Target,label
2,Target type,Multiclass
3,Original data shape,"(4295, 513)"
4,Transformed data shape,"(4295, 513)"
5,Transformed train set shape,"(3006, 513)"
6,Transformed test set shape,"(1289, 513)"
7,Numeric features,512
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


In [25]:
best_model = compare_models()

Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [26]:
# Create the final model
final_model = create_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3555,0.5417,0.3555,0.2796,0.2533,0.0488,0.0677
1,0.3488,0.5702,0.3488,0.4142,0.2547,0.0430,0.0571
2,0.3787,0.5974,0.3787,0.3659,0.2864,0.0895,0.1165
3,0.3621,0.5826,0.3621,0.2381,0.2552,0.0642,0.0863
4,0.3621,0.5563,0.3621,0.3231,0.2680,0.0639,0.0865
5,0.3289,0.5863,0.3289,0.2089,0.2262,0.0116,0.0163
6,0.3433,0.5339,0.3433,0.4019,0.2530,0.0352,0.0477
7,0.3567,0.5714,0.3567,0.3694,0.2522,0.0463,0.0659
8,0.3567,0.5389,0.3567,0.2638,0.2572,0.0537,0.0720


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [27]:
tuned_model = tune_model(final_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3588,0.5480,0.3588,0.2491,0.2063,0.0208,0.0682
1,0.3522,0.6037,0.3522,0.2067,0.2071,0.0136,0.0354
2,0.3555,0.5873,0.3555,0.2541,0.1995,0.0135,0.0556
3,0.3522,0.5587,0.3522,0.3519,0.2072,0.0192,0.0486
4,0.3522,0.5710,0.3522,0.2319,0.1967,0.0138,0.0525
5,0.3422,0.5888,0.3422,0.1713,0.1930,0.0030,0.0079
6,0.3500,0.5503,0.3500,0.2884,0.2064,0.0134,0.0361
7,0.3667,0.5806,0.3667,0.2485,0.2149,0.0315,0.0974
8,0.3500,0.5313,0.3500,0.2318,0.1977,0.0049,0.0173


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


# Validate model

In [ ]:
evaluate_model(tuned_model)